In [1]:
import requests
from bs4 import BeautifulSoup
import json
import re
import csv

In [2]:
url = "https://www.lix.polytechnique.fr/~hermann/conf.php"
response = requests.get(url)

In [3]:
def deadline_ahead_list(data):
  tbody = data.find('tbody')
  rows = tbody.find_all('tr')
  conference_array = []
  if len(rows) == 0: return conference_array
  for row in rows:
      cells = row.find_all('td')
      name = cells[0].find('a').text.strip() 
      conference_link = cells[0].find('a')['href']
      description = cells[0].find('span', class_='tooltiptext').text.strip() if cells[0].find('span', class_='tooltiptext') is not None else ""
      location = cells[1].text.strip()
      deadline = cells[2].text.strip()
      date = cells[3].text.strip()
      notification = cells[4].text.strip()
      subformat_details =re.sub(r'\s+', ' ',  cells[5].text.strip().replace("\n", "").replace("\t", ""))
      conference_array.append({'name':name, 'conference_link':conference_link,'description':description, 'location':location, 'deadline':deadline, 'date':date, 'notification':notification,'subformat_details':subformat_details})
  return conference_array

In [4]:
def running_list(data):
  tbody = data.find('tbody')
  rows = tbody.find_all('tr')
  conference_array = []
  if len(rows) == 0: return conference_array
  for row in rows:
      cells = row.find_all('td')
      name = cells[0].find('a').text.strip()
      conference_link = cells[0].find('a')['href']
      location = cells[1].text.strip()
      date = cells[2].text.strip()
      remark =re.sub(r'\s+', ' ',  cells[3].text.strip().replace("\n", "").replace("\t", ""))
      conference_array.append({'name':name, 'conference_link':conference_link,  'location':location, 'date':date, 'remark':remark})
  return conference_array

In [5]:
def deadline_over_list(data):
  tbody = data.find('tbody')
  rows = tbody.find_all('tr')
  conference_array = []
  if len(rows) == 0: return conference_array
  for row in rows:
    cells = row.find_all('td')
    name = cells[0].find('a').text.strip()
    conference_link = cells[0].find('a')['href']
    location = cells[1].text.strip()
    date = cells[2].text.strip()
    notification = cells[3].text.strip()
    final_version = cells[4].text.strip()
    early_registration = cells[5].text.strip()
    remarks = re.sub(r'\s+', ' ',  cells[6].text.strip().replace("\n", "").replace("\t", ""))
    conference_array.append({'name':name,'conference_link':conference_link,'location':location,'date':date,'notification':notification,'final_version':final_version,'early_registration':early_registration,'remarks':remarks})
  return conference_array

In [6]:
def planning_conference_list(data):
  tbody = data.find('tbody')
  rows = tbody.find_all('tr')
  conference_array = []
  if len(rows) == 0: return conference_array
  for row in rows:
    cells = row.find_all('td')
    name = cells[0].find('a').text.strip()
    conference_link = cells[0].find('a')['href']
    year = cells[1].text.strip()
    location = cells[2].text.strip()
    starting_date = cells[3].text.strip()
    ending_date = cells[4].text.strip()
    remarks = re.sub(r'\s+', ' ',  cells[5].text.strip().replace("\n", "").replace("\t", ""))
    conference_array.append({'name':name,'conference_link':conference_link,'year':year,'location':location,'starting_date':starting_date,'ending_date':ending_date,'remarks':remarks})
  return conference_array

In [7]:
deadline_ahead_conference_arr = []
running_conference_arr = []
deadline_over_future_conference_arr = []
planning_conference_arr = []
if(response.status_code == 200):
  soup = BeautifulSoup(response.content,'html.parser')
  tables = soup.find_all('table', class_="conference")
  deadline_ahead = tables[0]
  running = tables[1]
  deadline_over_future = tables[2] 
  planning_conference = tables[3]
  deadline_ahead_conference_arr = deadline_ahead_list(deadline_ahead)
  running_conference_arr = running_list(running)
  deadline_over_future_conference_arr = deadline_over_list(deadline_over_future)
  planning_conference_arr = planning_conference_list(planning_conference)
  

In [8]:
def write_json(data, filename):
    with open(filename, 'w') as file:
        json.dump(data, file, indent=4)

def write_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = data[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

In [9]:
write_json(deadline_ahead_conference_arr,"demo.json")
write_csv(deadline_ahead_conference_arr,"demo.csv")

In [12]:
import requests
import yaml
import json
import csv

In [11]:
# URL chứa dữ liệu YAML
url = "https://ccfddl.github.io/conference/allconf.yml"

# Tải nội dung từ URL
response = requests.get(url)
data = response.text

# Chuyển đổi từ YAML thành JSON
yaml_data = yaml.safe_load(data)
json_data = json.dumps(yaml_data, indent=4)

# Ghi vào tệp JSON
with open("allconf.json", "w") as json_file:
    json_file.write(json_data)

with open('allconf.yml', 'w') as file:
        try:
            yaml.dump(yaml_data, file)
        except yaml.YAMLError as e:
            print(e)

In [13]:
# Đọc nội dung từ tệp YAML
with open("allconf.yml", "r") as yaml_file:
    yaml_data = yaml.safe_load(yaml_file)
    
# Mở tệp CSV để ghi
with open("allconf.csv", "w", newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Viết header
    writer.writerow(["Title", "Description", "Sub", "Rank", "DBLP", "Year", "ID", "Link", "Abstract Deadline", "Deadline", "Timezone", "Date", "Place"])

    # Viết dữ liệu từ YAML vào CSV
    for conference in yaml_data:
        title = conference["title"]
        description = conference["description"]
        sub = conference["sub"]
        rank = conference["rank"]
        dblp = conference["dblp"]
        confs = conference["confs"]
        for conf in confs:
            year = conf["year"]
            conf_id = conf["id"]
            link = conf["link"]
            timeline = conf["timeline"][0]  # Lấy timeline đầu tiên trong trường hợp có nhiều timeline
            abstract_deadline = timeline.get("abstract_deadline", "")
            deadline = timeline.get("deadline", "")
            timezone = conf["timezone"]
            date = conf["date"]
            place = conf["place"]
            
            writer.writerow([title, description, sub, rank, dblp, year, conf_id, link, abstract_deadline, deadline, timezone, date, place])
print("Đã chuyển đổi thành công từ YAML thành CSV và lưu vào tệp allconf.csv")

Đã chuyển đổi thành công từ YAML thành CSV và lưu vào tệp allconf.csv
